In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
robot_raw = pd.read_csv('../../data/data_clean_0811.csv',index_col=0)

In [3]:
robot_raw = robot_raw.reset_index(drop=True)
data=robot_raw
data

,robot_id,rtc,protective_stop,task_state,rapid_rate,sum_running_time,ins_average_power,sum_running_circles,joint_actual_position,current_fluctuation,inst_temperature,inst_voltage,joint,health
0,Zu7250098,1628643756145,0,4,1.0,12589525,15.000000,3.016309e+05,51.366797,1.0,54.0,47.0,1,1
1,Zu3100161,1628643755723,0,4,1.0,12364110,3.400000,1.070656e+06,179.126535,1.5,46.0,48.0,1,1
2,Zu3100161,1628643752846,0,4,1.0,12364106,3.800000,1.070655e+06,249.018715,1.1,46.0,48.0,1,1
3,Zu7250098,1628643752716,0,4,1.0,12589521,36.400002,3.016302e+05,56.552356,0.8,54.0,48.0,1,1
4,Zu3100161,1628643749708,0,4,1.0,12364104,12.400000,1.070654e+06,134.857509,2.2,46.0,48.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537553,Zu3100091,1625448181050,0,4,1.0,12330690,0.300000,1.888191e+05,32.793401,0.8,50.0,47.0,6,1
3537554,Zu3100091,1625448179554,0,4,1.0,12330688,0.500000,1.888189e+05,54.094375,0.8,50.0,47.0,6,1
3537555,Zu3100091,1625448178687,0,4,1.0,12330688,0.100000,1.888189e+05,63.405684,0.3,50.0,47.0,6,1
3537556,Zu3100091,1625448177444,0,4,1.0,12330686,0.100000,1.888188e+05,20.751428,0.3,50.0,47.0,6,1


In [4]:
data.health.mean()

0.7687856990613299

In [5]:
data[data['health']==0].shape[0]/data.shape[0]

0.23121430093867013

In [6]:
data.columns

Index(['robot_id', 'rtc', 'protective_stop', 'task_state', 'rapid_rate',
       'sum_running_time', 'ins_average_power', 'sum_running_circles',
       'joint_actual_position', 'current_fluctuation', 'inst_temperature',
       'inst_voltage', 'joint', 'health'],
      dtype='object')

In [7]:
X = data
y = data['health']

In [8]:
# 从训练数据中分离验证集
X_train_full, X_valid_test_full, y_train, y_valid_test = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=42)
X_valid_full, X_test_full, y_valid ,y_test = train_test_split(X_valid_test_full,y_valid_test,train_size=0.5,test_size=0.5,random_state=42)

In [9]:
my_cols = [  'sum_running_time',
 'ins_average_power',
 'sum_running_circles',
 'joint_actual_position',
 'current_fluctuation',
 'inst_temperature']

In [10]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [11]:
my_model = XGBClassifier(n_estimators=500,random_state=42,learning_rate=0.1)
my_model.fit(X_train,y_train)

[09:20:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
prediction1 = my_model.predict(X_valid)

In [13]:
print(classification_report(y_valid, prediction1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     81573
           1       1.00      1.00      1.00    272183

    accuracy                           1.00    353756
   macro avg       1.00      1.00      1.00    353756
weighted avg       1.00      1.00      1.00    353756



In [14]:
prediction1

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [15]:
confusion_matrix(y_valid,prediction1, labels=None, sample_weight=None)

array([[ 81572,      1],
       [     1, 272182]], dtype=int64)

In [16]:
!dir

 Volume in drive D is LENOVO
 Volume Serial Number is E69E-F3CE

 Directory of D:\Desktop\JAKA\code\zdt_v1\ml\model\xgboost

2021/08/20  09:21    <DIR>          .
2021/08/20  09:21    <DIR>          ..
2021/08/20  09:21    <DIR>          .ipynb_checkpoints
2021/08/20  09:21            19,071 xgboost_0816_v1.0.ipynb
               1 File(s)         19,071 bytes
               3 Dir(s)  29,770,821,632 bytes free


In [17]:
import pickle 
#保存Model(注:save文件夹要预先建立，否则会报错)
with open('xgboost_0816_v1.0.pickle', 'wb') as f:
    pickle.dump(my_model, f)